In [1]:
from google.colab import drive
drive.mount('/content/drive')

# 주요 라이브러리 설치
!pip install mxnet
!pip install gluonnlp
!pip install transformers
!pip install sentencepiece

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Using cached gluonnlp-0.10.0.tar.gz (344 kB)
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595734 sha256=90ae456eba3afaca80c9d7fbbbe6b1af3c055cbe39e8c6b1d48b3985f82e1a7c
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
  Using cached transformers-4.18.0-py3-none-any.whl (4.0 MB)
     |████████████████████████████████| 6.5 MB 5.2 MB/s 
     |████████████████████████████████| 77 kB 5.8 MB/s 
     |████████████████████████████████| 596 kB 32.0 MB/s 
     |████████████████████████████████| 895 kB 44.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
import numpy as np
import os
import gc

import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# seed 값 설정
torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import transformers
from transformers import AdamW

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [3]:
from transformers import BertTokenizerFast, BertForSequenceClassification
# # 만약 colab pro가 아니면 model_name = 'bert-base-uncased'를 사용하세요
model_name = "bert-base-uncased"

tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
# device 설정
device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cuda:0


In [6]:
# 데이터 불러오기
path = '/content/drive/My Drive/Colab Notebooks/뉴스그룹분류/뉴스그룹분류경진대회/'
train = pd.read_csv(path+"/train.csv")

test = pd.read_csv(path+"/test.csv")
submission = pd.read_csv(path+"/sample_submission.csv")

In [7]:
import re 
# 숫자와 영문 빼고 모두 제거
def clean_text(texts): 
  corpus = [] 
  for i in range(0, len(texts)): 

    review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"\n\]\[\>]', '',texts[i]) #@%*=()/+ 와 같은 문장부호 제거
    review = re.sub(r'\s+', ' ', review) #extra space 제거
    review = re.sub(r'<[^>]+>','',review) #Html tags 제거
    review = re.sub(r'\s+', ' ', review) #spaces 제거
    review = re.sub(r"^\s+", '', review) #space from start 제거
    review = re.sub(r'\s+$', '', review) #space from the end 제거
    review = re.sub(r'_', ' ', review) #space from the end 제거
    corpus.append(review) 
  
  return corpus

In [8]:
temp = clean_text(train['text']) #메소드 적용
train['text'] = temp
train.head()

,id,text,target
0,0,They were and even if Washington might conside...,10
1,1,We run SpaceNews Views on our STAREACH BBS a l...,14
2,2,Not to worry The Masons have been demonized an...,19
3,3,Only Brendan McKay or maybe ARF would come to ...,17
4,4,Help I am running some sample problems from OR...,5


In [9]:
temp = clean_text(test['text']) #메소드 적용
test['text'] = temp
test.head()

,id,text
0,0,The VLIDE Adapter can be much faster then the ...
1,1,Yeah In a fire that reportedly burned hotter t...
2,2,Judge I grant you immunity from whatever may b...
3,3,I too put a corbin seat on my Hawk I got the s...
4,4,Do I ever After 2 years of having health probl...


In [10]:
from sklearn.model_selection import train_test_split
train_dataset, val_dataset = train_test_split(train, test_size = 0.1)
print(len(train_dataset))
print(len(val_dataset))

8309
924


In [11]:
# dataloader에서 오류가 나서 인덱스 재설정
train_dataset = train_dataset.reset_index(drop=True)
val_dataset = val_dataset.reset_index(drop=True)

In [12]:
# train, val에 사용
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):

        # 데이터프레임 칼럼 들고오기
        sentence1 = self.df_data.loc[index, 'text']

        encoded_dict = tokenizer.encode_plus(
                    sentence1,
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    truncation=True,
                    return_attention_mask = True,   
                    return_tensors = 'pt',          
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
        # 숫자로 변환된 label을 텐서로 변환
        target = torch.tensor(self.df_data.loc[index, 'target'])
        # input_ids, attention_mask, label을 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask, target)

        return sample

    def __len__(self):
        return len(self.df_data)
    
# test 예측에 사용
class TestDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):

        sentence1 = self.df_data.loc[index, 'text']


        encoded_dict = tokenizer.encode_plus(
                    sentence1, 
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    return_attention_mask = True,   
                    truncation=True,
                    return_tensors = 'pt',          
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        # input_ids, attention_mask를 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask)

        return sample


    def __len__(self):
        return len(self.df_data)

In [13]:
# 모델 하이퍼파라미터

NUM_EPOCHS = 3
L_RATE = 1e-5 
MAX_LEN = 512

TRAIN_BATCH_SIZE = 4 # batch size가 클수록 global minimum에 도달하는 속도가 증가합니다. (GPU 메모리에 따라 변경해 주세요, 너무 크면 OOM 문제가 발생합니다.)
TEST_BATCH_SIZE = 1

NUM_CORES = os.cpu_count() # Dataloader에 사용됩니다. 

NUM_CORES

2

In [14]:
train_data = CompDataset(train_dataset)
val_data = CompDataset(val_dataset)
test_data = TestDataset(test)

# batch_size 만큼 데이터 분할
train_dataloader = DataLoader(train_data,
                                batch_size=TRAIN_BATCH_SIZE,
                                shuffle=True,
                                num_workers=NUM_CORES)

val_dataloader = DataLoader(val_data,
                            batch_size=TRAIN_BATCH_SIZE,
                            shuffle=True,
                            num_workers=NUM_CORES)

test_dataloader = DataLoader(test_data,
                                batch_size=TEST_BATCH_SIZE,
                                shuffle=False,
                                num_workers=NUM_CORES)



print(len(train_dataloader))
print(len(val_dataloader))
print(len(test_dataloader))

2078
231
9233


In [15]:
model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels = 20, # 출력 label의 개수
)

# model을 device위에 올림
model.to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [16]:
# optimizer 설정
optimizer = AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8 
            )

In [17]:
loss_fn = torch.nn.CrossEntropyLoss()

# loss값 저장
loss_values = []

# 학습 시작
for epoch in range(NUM_EPOCHS):
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))
    
    stacked_val_labels = []
    targets_list = []

    # ========================================
    #               Training
    # ========================================
    
    print('Training...')
    
    # train mode 변환
    model.train()
    # True로 설정하게 되면 해당 텐서에서 어떤 연산이 이루어졌는지 추적할 수 있고, 해당 텐서에 대한 그라디언트를 저장하게 됩니다. 
    torch.set_grad_enabled(True)


    # 1epoch마다 loss값 초기화
    total_train_loss = 0

    for i, batch in enumerate(tqdm(train_dataloader)):
        
        train_status = 'Batch ' + str(i) + ' of ' + str(len(train_dataloader))
        
        print(train_status, end='\r')


        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        # 3개의 인풋
        outputs = model(b_input_ids, 
                    attention_mask=b_input_mask)
        
        # outputs tuple: (loss, logits)
        loss = loss_fn(outputs[0], b_labels)
        
        # loss는 텐서이기 때문에 숫자로 변환 후 더합니다. 
        total_train_loss = total_train_loss + loss.item()
        
        # backward()를 하기 전에 optimizer의 그라디언트를 0으로 합니다. 
        optimizer.zero_grad()
        
        # 그라디언트 계산
        loss.backward()
        
        
        # "exploding gradients" 문제를 예방해줍니다.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        
        # optimizer 가중치 업데이트
        optimizer.step() 
    
    print('Train loss:' ,total_train_loss)


    # ========================================
    #               Validation
    # ========================================
    
    print('\nValidation...')

    # evaluation mode로 변환
    model.eval()

    # validation 과정에서는 그라디언트를 연산하거나 저장하지 않습니다.(메모리, 진행 속도 세이브)
    torch.set_grad_enabled(False)
    
    total_val_loss = 0
    

    for j, batch in enumerate(tqdm(val_dataloader)):
        
        val_status = 'Batch ' + str(j) + ' of ' + str(len(val_dataloader))
        
        print(val_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)      


        outputs = model(b_input_ids, 
                attention_mask=b_input_mask)
        
        # 예측값
        preds = outputs[0]

        loss = loss_fn(preds, b_labels)
        total_val_loss = total_val_loss + loss.item()
        

        # 예측값을 CPU로 이동시킵니다. 
        val_preds = preds.detach().cpu().numpy()
        
        # labels을 cpu로 이동시킵니다.
        targets_np = b_labels.to('cpu').numpy()

        targets_list.extend(targets_np)

        if j == 0:  # 첫 번째 batch일 떄
            stacked_val_preds = val_preds

        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))
        

    
    # validation accuracy 계산
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)
    
    val_acc = accuracy_score(y_true, y_pred)
    
    
    print('Val loss:' ,total_val_loss)
    print('Val acc: ', val_acc)


    # 모델 저장
    torch.save(model.state_dict(), 'epoch:{}_model.pt'.format(epoch))
    
    # 메모리 관리
    gc.collect()


======== Epoch 1 / 3 ========
Training...


  0%|          | 0/2078 [00:00<?, ?it/s]

  0%|          | 1/2078 [00:01<38:45,  1.12s/it]

  0%|          | 2/2078 [00:01<32:04,  1.08it/s]

  0%|          | 3/2078 [00:02<30:02,  1.15it/s]

...

  2%|▏         | 43/2078 [00:34<27:01,  1.26it/s]

  2%|▏         | 44/2078 [00:35<26:57,  1.26it/s]

  2%|▏         | 45/2078 [00:35<27:00,  1.25it/s]

  2%|▏         | 45/2078 [00:36<27:33,  1.23it/s]


KeyboardInterrupt: ignored

In [ ]:
# Val loss: 221.98762597795576
# Val acc:  0.6991341991341992

# Val loss: 248.69036069093272
# Val acc:  0.724025974025974

# Val loss: 298.04990690317936
# Val acc:  0.7337662337662337

In [ ]:
# test 시작
for j, batch in enumerate(tqdm(test_dataloader)):
        
        inference_status = 'Batch ' + str(j+1) + ' of ' + str(len(test_dataloader))
        
        print(inference_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)

        outputs = model(b_input_ids, 
                attention_mask=b_input_mask)
        
        # 예측값
        preds = outputs[0]
        preds = preds.detach().cpu().numpy()

        
        if j == 0:  
            stacked_preds = preds

        else:
            stacked_preds = np.vstack((stacked_preds, preds))

In [ ]:
stacked_preds

In [ ]:
# argmax로 label결정
preds = np.argmax(stacked_preds, axis=1)

preds

In [ ]:
submission['target']=preds

In [ ]:
submission.to_csv("sample_submission.csv", index = False)

In [ ]:
# 파일 저장
from google.colab import files
files.download("sample_submission.csv")